In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'/opt/spark-2.4.5'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
from datetime import datetime
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize']= 20,20
%matplotlib inline
import numpy as np
import pandas as pd
 
 

In [3]:
spark = SparkSession.builder.master("local").appName("app name").config("spark.some.config.option").getOrCreate()

In [4]:
spark = SparkSession \
.builder \
.appName("GPSSpark") \
.getOrCreate()

In [5]:
s=spark.sparkContext

In [6]:
#points received on one day
sql_c=SQLContext(s)
df= sql_c.read.orc('/FYP/huntersfyp17/date=2019-11-11/part*')


In [7]:
df.dropDuplicates(['driver_id']).show()

+--------------------+------+----------+-------------+-------------+-------------+
|           driver_id|status| timestamp|     latitude|    longitude|vehicle_model|
+--------------------+------+----------+-------------+-------------+-------------+
|01c4a09a7ac18a6dd...|     F|1573439175|6.92770862579|79.9471359253|            3|
|032d1422fa4881996...|     F|1573446616|6.69899654388|79.9615325928|            1|
|05a2444bd5286fe06...|     F|1573460778|6.70115375519|79.9124603271|            3|
|060f1b726d097c355...|     F|1573446921|6.83433580399|79.9345474243|            1|
|0e45899a6c083f3b4...|     F|1573458360|7.05800199509|79.9316940308|            1|
|0ebc7bbafaa0e8bcd...|     F|1573443977|6.82634162903|80.0523757935|            2|
|17d8a0bde28684ec5...|     F|1573447231|6.88117933273|79.9154510498|            2|
|19c8911fd4a96eff5...|     F|1573434000|6.81323194504|80.1042098999|            1|
|24e3ef295497fb46b...|     F|1573468388|7.15361499786| 79.881942749|            1|
|33b

In [14]:
print(df.count())

30984629


In [8]:
gps_df=df.dropDuplicates(['driver_id'])

In [15]:
print(gps_df.count())

14140


In [17]:
driver_vehicle_model=[]
def getDriverGPSpoints(driverid):
    driver_gps=gps_df.select(["driver_id","timestamp","latitude","longitude","vehicle_model"]).where(gps_df.driver_id == driverid ).sort("timestamp").collect()    
    columns2=["driver_id","timestamp","latitude","longitude","vehicle_model"]
    df_gps = pd.DataFrame(driver_gps, columns=columns2)
    if (df_gps.empty):
        driver_vehicle_model.append("")
    else:
        driver_vehicle_model.append(df_gps['vehicle_model'].unique())
    return driver_vehicle_model

In [18]:
drivers=pd.read_csv('driver.csv')

In [19]:
drivers['vehicle_model']=""

In [20]:
drivers

,driver_id,vehicle_model
0,0038263cc90dfa55d621a6425e46020136a318d12b9743...,
1,006ed08c7fe1278c64a3c5dbac78c7b0a9cec8d7316c51...,
2,00c6dda371d7192310a3b12f05c8c82d56e1b7a39537c2...,
3,00ce195f11cfa5d5e228095893a866e2ec79cf3aa1c423...,
4,010cbc4ccc307d3b4fd7214c8d6c76d700ec0561fc467a...,
...,...,...
1070,9fee5fbbacce56ba60b43fa90e8b83692d0c30e2c77766...,
1071,a32618a8be30c4b28ff8376cd0d0e907fefb0449f19f14...,
1072,a6e6684c7a8bc8f4d7b74a4a17ebed8eeab7474bf40db8...,
1073,add78008bf890a56b09dccf59a6612b00121044b22aefa...,


In [ ]:
count =0
vehicle_model=[]
for row in drivers.to_records(index=False):
    count+=1
    print(count)
    driver=getDriverGPSpoints(row['driver_id'])
    vehicle_model.append(driver[0])
    #print(vehicle_model)
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
textfile = open("vehicle_model.txt", "w")
for element in vehicle_model:
    textfile.write(element + "\n")
textfile.close()